In [ ]:
import sys

# Set the project path here
PathProj = 
sys.path.append(PathProj)

from genDispatch import *
from ToWRF import *

<h3><i><font color=#2471A3>Module-level names</font></i></h3>

In [ ]:
# Dictionary of the input fundamentals. Objects bound are datahandler.Metadata instances
dFundamentals['sys demand']

In [ ]:
# Inspect the underlying data via the .val attr
dFundamentals['sys demand'].val

In [ ]:
# This contains the read parameters. This particular parameter is the MWh HHV per market unit of the fuels
dParameters['fuel qty to MWh']

In [ ]:
# Inspect the scenarios set with this. This dictionary is read-only
Scenarios

In [ ]:
# The initialized GenUnit fleet from PowerPlants.py, as read from the power plant database
GenFleet

In [ ]:
# Following OOP, all the info about each unit is contained the object
GenFleet[0].gentech

In [ ]:
# Read power plant database. Each sheet is contained here.
PPdb['master']

<h3><i><font color=#117A65>1) Set the simulation period and the scenarios</font></i></h3>

In [ ]:
# if no arguments are passed, then the full common period is set
set_period()
#set_period(t_from='2016 Apr 01', t_to='2016 Apr 01')

# Set the system demand to the baseline
set_scenario({'sys demand': 'Baseline'})

<h3><i><font color=#117A65>2) Simulate dispatch</font></i></h3>

In [ ]:
# simulate()'s docstring in NEMS.py documents the data objects initially written to the results (contains the dispatch).
# After a run, it's recommended to save the results by pickling them.
Results = solve(simulation_name='Baseline', runchecks=False, daily_progress=True)


# calc_allheat() calculates statistics related to waste and cogeneration heat, some of which is needed by 
# plot_ElecAndHeatByPlant() and in parts of Step 4
calc_allheat(Results)

<h3><i><font color=#117A65>3) Analyze dispatch results</font></i></h3>

In [ ]:
# gd_analysis.py contains a number of analysis and plotting functions, such as the fuel mix plotter:
# The generated plot data is written in Results as well
plot_fuelmix(Results)

In [ ]:
plot_ElecAndHeatByPlant(Results)

<h3><i><font color=#117A65>4) Calculate heat streams for WRF</font></i></h3>
genDispatch does not automatically calculate the heat stream components of the waste heat. These functions are provided in ToWRF.py

In [ ]:
# Thermal analysis is done to calculate the how the fuel HHV is allocated into the electricity, cogen heat, and waste heat streams.
# Tolerate CCGT warnings (will adj Rankine eff)
compiled_prms = thermal_analysis(GenFleet, dParameters)

In [ ]:
# Calculate the waste heat streams. First by outlet, then by kind.
calc_heatstreams(GenFleet, Results, by='outlet',)
calc_heatstreams(GenFleet, Results, by='kind', latentheatfactors=dParameters['latent heat factors'])

In [ ]:
# Prepare the WRF input files
WRF_SH, WRF_LH, WRF_Sea, total_MWh = prep_WRF_inputs(
    scenario='Test WRF', 
    GenFleet=GenFleet, 
    PPcells_only=False,   # Set to False if you want the full grid (sparse)
    unit='MW',
    day='2016 Apr 01', 
    write_to_disk=False,
)

In [ ]:
total_MWh * c_['MWh to ktoe']

In [ ]:
WRF_SH

SCRATCH